# 결측치 처리 전략: 시계열 보간 → 공간 보간 → 인접 관측소 평균 대체

# 라이브러리 불러오기

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import requests
import json

# 데이터 불러오기

In [2]:
url1 = "https://raw.githubusercontent.com/s0nghyunje0ng/2025-weather-bigdata-contest/safety/topic2_safety/data/raw/call119_train.csv"
url2 = "https://raw.githubusercontent.com/s0nghyunje0ng/2025-weather-bigdata-contest/safety/topic2_safety/data/raw/cat119_train.csv"

df_call = pd.read_csv(url1)
df_cat = pd.read_csv(url2)

# 데이터 확인

In [3]:
print(df_call.shape)
df_call.head()

(42924, 14)


,call119_train.tm,call119_train.address_city,call119_train.address_gu,call119_train.sub_address,call119_train.stn,call119_train.ta_max,call119_train.ta_min,call119_train.ta_max_min,call119_train.hm_min,call119_train.hm_max,call119_train.ws_max,call119_train.ws_ins_max,call119_train.rn_day,call119_train.call_count
0,20200501,부산광역시,강서구,대저2동,904,23.7,16.6,7.1,63.2,90.3,5.8,10.6,0.0,1
1,20200501,부산광역시,강서구,천성동,921,24.3,15.4,8.9,64.2,96.4,4.8,8.6,0.0,1
2,20200501,부산광역시,금정구,금사동,940,25.6,17.2,8.4,46.6,76.4,4.8,8.3,0.0,1
3,20200501,부산광역시,금정구,금성동,941,25.3,17.1,8.2,-99.0,-99.0,7.1,9.7,0.0,1
4,20200501,부산광역시,금정구,남산동,939,26.5,16.7,9.8,-99.0,-99.0,5.7,8.1,0.0,3


In [4]:
df_call.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42924 entries, 0 to 42923
Data columns (total 14 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   call119_train.tm            42924 non-null  int64  
 1   call119_train.address_city  42924 non-null  object 
 2   call119_train.address_gu    42924 non-null  object 
 3   call119_train.sub_address   42924 non-null  object 
 4   call119_train.stn           42924 non-null  int64  
 5   call119_train.ta_max        42924 non-null  float64
 6   call119_train.ta_min        42924 non-null  float64
 7   call119_train.ta_max_min    42924 non-null  float64
 8   call119_train.hm_min        42924 non-null  float64
 9   call119_train.hm_max        42924 non-null  float64
 10  call119_train.ws_max        42924 non-null  float64
 11  call119_train.ws_ins_max    42924 non-null  float64
 12  call119_train.rn_day        42924 non-null  float64
 13  call119_train.call_count    429

In [5]:
print(df_cat.shape)
df_cat.head()

(61771, 8)


,cat119_train.tm,cat119_train.address_city,cat119_train.address_gu,cat119_train.sub_address,cat119_train.cat,cat119_train.sub_cat,cat119_train.stn,cat119_train.call_count
0,20200501,부산광역시,강서구,대저2동,구급,교통사고,904,1
1,20200501,부산광역시,강서구,천성동,구조,교통사고,921,1
2,20200501,부산광역시,금정구,금사동,구급,부상,940,1
3,20200501,부산광역시,금정구,금성동,기타,업무운행,941,1
4,20200501,부산광역시,금정구,남산동,구급,부상,939,1


In [6]:
df_cat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61771 entries, 0 to 61770
Data columns (total 8 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   cat119_train.tm            61771 non-null  int64 
 1   cat119_train.address_city  61771 non-null  object
 2   cat119_train.address_gu    61771 non-null  object
 3   cat119_train.sub_address   61771 non-null  object
 4   cat119_train.cat           61771 non-null  object
 5   cat119_train.sub_cat       61771 non-null  object
 6   cat119_train.stn           61771 non-null  int64 
 7   cat119_train.call_count    61771 non-null  int64 
dtypes: int64(3), object(5)
memory usage: 3.8+ MB


# 컬럼명 정리

컬럼명에서 불필요한 접두어 제거

In [7]:
df_call.columns = [col.replace("call119_train.", "") for col in df_call.columns]
df_cat.columns = [col.replace("cat119_train.", "") for col in df_cat.columns]

두 데이터프레임의 `call_count`가 충돌하지 않도록 명확화

In [8]:
df_cat.rename(columns={"call_count": "sub_call_count"}, inplace=True)

# 불필요한 컬럼 제거 및 수정

모든 데이터에 같은 값(부산광역시)을 가지는 `address_city` 제거

In [9]:
df_call.drop(columns=["address_city"], inplace=True)
df_cat.drop(columns=["address_city"], inplace=True)

동명이동(송정동)이 있어서 군/구와 읍/면/동을 합쳐 명확화

In [10]:
df_call["sub_address"] = df_call["address_gu"].astype(str) + " " + df_call["sub_address"].astype(str)
df_cat["sub_address"] = df_cat["address_gu"].astype(str) + " " + df_cat["sub_address"].astype(str)

# 변수 타입 변환

- 시간 기반 분석 및 파생 변수 생성을 위함

    - `tm`(날짜): `int` → `datetime`

- 메모리 절약과 연산 효율성 향상을 위함

    - `stn`(AWS 지점 코드): `int` → `category`

    - `address_gu`(군/구명): `object` → `category`

    - `sub_address`(읍/면/동명): `object` → `category`

    - `cat`(신고 분류(대분류)): `object` → `category`

    - `sub_cat`(신고 분류(소분류)): `object` → `category`

In [11]:
df_call["tm"] = pd.to_datetime(df_call["tm"], format="%Y%m%d")
df_call["stn"] = df_call["stn"].astype("category")
df_call["address_gu"] = df_call["address_gu"].astype("category")
df_call["sub_address"] = df_call["sub_address"].astype("category")

df_cat["tm"] = pd.to_datetime(df_cat["tm"], format="%Y%m%d")
df_cat["stn"] = df_cat["stn"].astype("category")
df_cat["address_gu"] = df_cat["address_gu"].astype("category")
df_cat["sub_address"] = df_cat["sub_address"].astype("category")
df_cat["cat"] = df_cat["cat"].astype("category")
df_cat["sub_cat"] = df_cat["sub_cat"].astype("category")

# 결측치 처리 ⭐⭐⭐

image.png

In [12]:
df_call.replace(-99.0, np.nan, inplace=True)

In [13]:
missing_cols = df_call.isna().sum()
missing_cols[missing_cols > 0]

ta_max         133
ta_min         133
ta_max_min     133
hm_min        3056
hm_max        3056
ws_max         141
ws_ins_max     141
rn_day         255
dtype: int64

## B. 시계열 보간 → 공간 보간 전략

- 먼저 관측소 기준으로 시간 흐름에 따라 선형 보간 ← 시간 흐름 패턴을 반영

- 그 후 남은 결측은 같은 행정구(`address_gu`)의 평균값으로 결측을 채움 ← 지역적 유사성 활용

In [14]:
df_b = df_call.drop(columns=["ta_max_min"]).copy()

# 보간 대상 컬럼
target_cols = ["ta_max", "ta_min", "hm_max", "hm_min", "ws_max", "ws_ins_max", "rn_day"]

# -----------------------------------
# 행정구별 날짜 단위 평균 테이블 생성
# -----------------------------------
gu_to_stns = df_call.groupby("address_gu", observed=False)["stn"].unique().to_dict()

avg_weather_by_gu_tm = []

for gu, stns in gu_to_stns.items():
    subset = df_call[df_call["stn"].isin(stns)].copy()
    group_avg = subset.groupby("tm", observed=False)[target_cols].mean()
    group_avg["address_gu"] = gu
    avg_weather_by_gu_tm.append(group_avg.reset_index())

df_avg_weather = pd.concat(avg_weather_by_gu_tm, ignore_index=True)

# ------------------------
# [B-1] 공간 → 시계열 보간
# ------------------------

# 시계열 보간
df_b = df_b.sort_values(["stn", "tm"])
for col in target_cols:
    df_b[col] = df_b.groupby("stn", observed=False)[col].transform(lambda x: x.interpolate())

# 보간 후 결측치 현황 확인
print(f"시계열 보간 후 남은 결측:\n{df_b[target_cols].isna().sum()}")

# 공간 보간
for col in target_cols:
    is_missing = df_b[col].isna()
    for idx in df_b[is_missing].index:
        row = df_b.loc[idx]
        gu = row["address_gu"]
        date = row["tm"]

        # 평균 테이블에서 해당 구 + 날짜의 값 추출
        match = df_avg_weather[(df_avg_weather["address_gu"] == gu) & (df_avg_weather["tm"] == date)]

        if not match.empty:
            fill_value = match[col].values[0]
            if not pd.isna(fill_value):
                df_b.at[idx, col] = fill_value

# 보간 후 결측치 현황 확인
print(f"\n공간 보간 후 남은 결측:\n{df_b[target_cols].isna().sum()}")

# ----------------------
# [B-2] 인접 관측소 평균
# ----------------------

# 인접 관측소 딕셔너리
response = requests.get("https://raw.githubusercontent.com/s0nghyunje0ng/2025-weather-bigdata-contest/safety/topic2_safety/data/processed/busan_stn_neighbors.json")
stn_neighbors_dict = json.loads(response.text)
stn_neighbors_dict = {int(k): v for k, v in stn_neighbors_dict.items()}

# 인접 관측소 평균으로 대체
for col in target_cols:
    is_missing = df_b[col].isna()
    for idx in df_b[is_missing].index:
        row = df_b.loc[idx]
        stn = row["stn"]
        date = row["tm"]
        neighbors = stn_neighbors_dict.get(stn, [])
        if neighbors:
            neighbor_values = df_b[(df_b["tm"] == date) & (df_b["stn"].isin(neighbors))][col].dropna()
            if len(neighbor_values) > 0:
                df_b.at[idx, col] = neighbor_values.mean()

# 보간 후 결측치 현황 확인
print(f"\n인접 관측소 평균 대체 후 남은 결측:\n{df_b[target_cols].isna().sum()}")

시계열 보간 후 남은 결측:
ta_max           0
ta_min           0
hm_max        2918
hm_min        2918
ws_max           0
ws_ins_max       0
rn_day           0
dtype: int64

공간 보간 후 남은 결측:
ta_max           0
ta_min           0
hm_max        1462
hm_min        1462
ws_max           0
ws_ins_max       0
rn_day           0
dtype: int64

인접 관측소 평균 대체 후 남은 결측:
ta_max        0
ta_min        0
hm_max        0
hm_min        0
ws_max        0
ws_ins_max    0
rn_day        0
dtype: int64


# 전처리된 데이터 저장

In [ ]:
# 일교차 계산
df_b["ta_max_min"] = df_b["ta_max"] - df_b["ta_min"]

len(df_b[df_b["ta_max_min"] < 0])

0

In [ ]:
df_b = df_b[["tm", "address_gu", "sub_address", "stn", "ta_max", "ta_min", "ta_max_min", "hm_max", "hm_min", "ws_max", "ws_ins_max", "rn_day", "call_count"]]
df_b.to_csv("preprocessed_B_temporal_first.csv", index=False)